In [ ]:
import shlex
import re
import os
from pprint import pprint
import subprocess
import pandas as pd
import yaml

import ipywidgets as widgets
from ipywidgets import interact

In [ ]:
with open("config_postprocessing_gtex_v7.yaml") as config_f:
    config = yaml.load(config_f)      

### Models and covariances

In [ ]:
model_regex = re.compile(config['model_regex'])
covariances_regex = re.compile(config['covariances_regex'])

In [ ]:
models = [x for x in os.listdir(config['model_db_folder']) if x.endswith("db")]
models = [os.path.join(config['model_db_folder'], x) for x in models]
tissues = [model_regex.match(model).group(1) for model in models]
models = dict(zip(tissues, models))

In [ ]:
covariances = [x for x in os.listdir(config['model_db_folder']) if x.endswith("txt.gz")]
covariances = [os.path.join(config['model_db_folder'], x) for x in covariances]
tissues = [covariances_regex.match(covariance).group(1) for covariance in covariances]
covariances = dict(zip(tissues, covariances))

### Phenotypes
One phenotype is defined by the ID of a execution (a timestamp) and the name of a latent variable.

In [ ]:
latent_variables = ["z" + str(i) for i in range(8)]
phenotypes = [(run, z) for run in config['runs'] for z in latent_variables]

In [ ]:
gwas_files = { phenotype: config['gwas_pattern'].format(*phenotype) for phenotype in phenotypes }

In [ ]:
spredixcan_output_pattern = os.path.join(config['gwas_folder'], config['spredixcan_output_pattern'])

### Harmonization

In [ ]:
gwas_harmon_pattern = "{}/test_std_covariates_PC__GBR_unrelated__qc/BGEN/GWAS__{}__harmonized__test_std_covariates_PC__GBR_unrelated__qc.tsv"
gwas_harmon_files = { phenotype: gwas_harmon_pattern.format(*phenotype) for phenotype in phenotypes }

In [ ]:
REF_DATA_ARGS = [
    # "-snp_reference_metadata", " %s METADATA" % VARIANT_METADATA, 
    "-liftover", config['CHAIN_FILE']
]

HARMON_COLMAPPING_OPTIONS = [
    '-output_column_map', 'SNP', 'variant_id', 
    '-output_column_map', 'a_1', 'effect_allele',
    '-output_column_map', 'a_0', 'non_effect_allele',
    '-output_column_map', 'BP', 'position',
    '-output_column_map', 'CHR', 'chromosome', '--chromosome_format', 
    '-output_column_map', 'af', 'frequency', 
    '-output_column_map', 'P', 'pvalue',
    '-output_column_map', 'BETA', 'effect_size',
    '-output_column_map', 'SE', 'standard_error'    
]

OUTPUT_ORDER = [ 
    '-output_order', 
    'variant_id', 
    'panel_variant_id', 
    'chromosome', 
    'position', 
    'effect_allele', 
    'non_effect_allele', 
    'frequency', 
    'pvalue', 
    'zscore', 
    'effect_size', 
    'standard_error' 
]

Select phenotypes:

In [ ]:
runs_w = widgets.SelectMultiple(options=config['runs']); display(runs_w)
z_w = widgets.SelectMultiple(options=latent_variables); display(z_w)

Assemble harmonization commands:

In [ ]:
harmon_commands = {}

for run in runs_w.value:
    for z in z_w.value:
    
        gwas_file = os.path.join(gwas_folder, gwas_files[(run, z)])
        gwas_harmon_file = os.path.join(gwas_folder, gwas_harmon_files[(run, z)])

        harmon_command = [ 'python', PARSING_SCRIPT ]
        harmon_command.extend([ '-gwas_file', gwas_file, '-output', gwas_harmon_file ])                
        harmon_command.extend(REF_DATA_ARGS)
        harmon_command.extend(HARMON_COLMAPPING_OPTIONS)
        harmon_command.extend(OUTPUT_ORDER)        
        
        harmon_commands[(run, z)] = harmon_command

Save individual harmonization commands as lines in a sh file:

In [ ]:
with open("harmonization_commands.sh", "w") as script:
    script.write("#!/bin/bash\n")
    for phenotype in harmon_commands:
        dd = harmon_commands[phenotype]
        harmon_command = "%s\n\n" % " ".join(dd)
        script.write(harmon_command)

In [ ]:
# pprint(harmon_commands)

### S-PrediXcan configurations

In [ ]:
GWAS_COLUMNS = {
  "--snp_column": "SNP",
  "--effect_allele_column": "a_1",
  "--non_effect_allele_column": "a_0",
  "--beta_column": "BETA",
  "--pvalue_column": "P"
}     

In [ ]:
tissues_w = widgets.SelectMultiple(options=sorted(list(covariances.keys()))); display(tissues_w)

Assemble SPrediXcan commands:

In [ ]:
arguments = {}

for tissue in tissues_w.value:
    
    arguments[tissue] = {}
    
    for run in runs_w.value:
        for z in z_w.value:
    
            gwas_file = os.path.join(gwas_folder, gwas_files[(run, z)])
        
            args = {
               "--model_db_path": models[tissue],
               "--covariance": covariances[tissue],
               "--gwas_folder": os.path.dirname(gwas_file),
               '--gwas_file_pattern': os.path.basename(gwas_file),
               "--output_file": spredixcan_output_pattern.format(run, run, z, tissue),
            }
            
            args.update(GWAS_COLUMNS)
            arguments[tissue][(run, z)] = args

In [ ]:
def to_command(t):
    t = list(t.items())
    t = [item for sublist in t for item in sublist]
    spredixcan_command = [spred_exec] + t
    spredixcan_command = " ".join(spredixcan_command)
    return spredixcan_command

Add SPrediXcan commands to sh file

In [ ]:
with open("spred_commands.sh", "w") as script:
    script.write("#!/bin/bash\n")
    for tissue in arguments:
        for phenotype in arguments[tissue]:
            spredixcan_command = to_command(arguments[tissue][phenotype])
            script.write(spredixcan_command + "\n")

In [ ]:
# tissues_w = widgets.Dropdown(options=sorted(list(covariances.keys()))); display(tissues_w)
# runs_w = widgets.Dropdown(options=config['runs']); display(runs_w)
# z_w = widgets.Dropdown(options=[latent_variables]); display(z_w)

## Transcriptome-wide association studies (TWAS)

I ran the S-PrediXcan tool on my GWAS results and gene expression prediction models, to find associations between imputed gene expression levels and the latent variables.

In [ ]:
def f(runs_w, z_w, tissues_w):
    spred_results = spredixcan_output_pattern.format(runs_w, runs_w, z_w, tissues_w)
    df = pd.read_csv(spred_results)
    
    display(
        df.drop(["pred_perf_qval", "pred_perf_qval", "n_snps_used", "n_snps_in_cov", "n_snps_in_model"], axis=1)
    )
    
kk = interact(f, runs_w=runs_w,  z_w=z_w, tissues_w=tissues_w)